## Goals:

### Gradient Statistics:
- Layerwise/Modulewise Norm, Varaince
- Gradient Variance/Mean for same samples with different augmentation
- Gradient after each Epoch on same Task
- Gradient over the corse of training a full network
- Can we use the Data statistics of the augmented gradients to clustet examples ? 

### Observation:
- Does a high loss mean we get high mean gradient values: 
    - Exp take full dataset. 
    - Augment each sample 10 times
    - Compute statistics
    - Logg also mean and variance value
    - Compute scatter plot of this
        
- Compare the variance of data augmentation to the full dataset:
    - We assume the variance of the dataset should be largern than the variance of the sample sample
    
- Can we observe that after the first layer the network gets invariant to for example color jolor jit ? 
    
    
### Where we should have a closer look:
- Is there a way to compare samples based on there loss function ? 
    - Benefits:
        - Feature Maps are not invariant to translation (Therefore we used the variation where we assosicated a feature vector to a label). If we look at the loss induced on the parameters ths should be more sophisticated ?
        - If two samples would change the model in the same ways -> They might contain the same constraints
        - Better metric for measuring similarity
        
- So what we currently do when we replay samples is to force the weight update to include the gradient induced by the replay sampled within the memory. 
- Can we compute the gradient update over mutiple samples and store them as a "Key Direction in Weight Space for a given Task" or are we losing a lot of desirable properties of SGD by doing so ? This GradientAccumulation could we added when learning a new task. 
- Will this stay fixed over a lot of update steps ?


# GEM:
- Store samples in memory
- Compute gradient over each element in memory.
- Solve LQ-Programm to project gradient st. when NN is assumed to be linear no constraint is violated. 
- New gradient smallest distance to "original" gradient based on L2 Norm non violating the constraint.

- For each update requires gradient computation over full memory.

# A-GEM:
- Store samples in memory
- Compute gradient of memory batch
- Compute gradient of new samples
- Apply update rule based on inner product of new and old batch (simple gradient projection)

- For each update requires gradient comutation over batch in memory.


# Proposed: 
- Put samples into memory based on gradient (keep most restrictive ones around)
- Can we keep the gradient fixed over training for an epoch for small batch sizes ?
- Can we compute a desired "Weight" for each sample in the memory ? Apply GD-Times for a Single Sample until good loss achieved -> store the resulting weight as a target weight for this Sample ? 


In [ ]:

# gradient statistic -> over full dataset
# For each weight -> mean and std.

import os
import sys 
os.chdir(os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.insert(0, os.path.join(os.getenv('HOME'), 'ASL'))
sys.path.append(os.path.join(os.path.join(os.getenv('HOME'), 'ASL') + '/src'))

from torchvision import transforms as tf
import numpy as np
import pickle
import torch
import torch.nn.functional as F
from utils_asl import file_path, load_yaml
from models_asl import FastSCNN

In [ ]:
# SETUP MODEL

eval_cfg_path="cfg/eval/eval.yml"
env_cfg_path = os.path.join('cfg/env', os.environ['ENV_WORKSTATION_NAME']+ '.yml')
env_cfg = load_yaml(env_cfg_path)	
eval_cfg = load_yaml(eval_cfg_path)
device= "cuda"
model = FastSCNN(**eval_cfg['model']['cfg'])


p = os.path.join( env_cfg['base'], eval_cfg['checkpoint_load'])
print(p)

def load(model, p):
    if os.path.isfile( p ):
        res = torch.load(p)
        new_statedict = {}
        for k in res['state_dict'].keys():
            if k.find('model.') != -1: 
                new_statedict[ k[6:]] = res['state_dict'][k]
        res = model.load_state_dict( new_statedict, strict=True)
        print('Restoring weights: ' + str(res))
    else:
        raise Exception('Checkpoint not a file')
    return model
model = load(model,p)

model.to(device)

In [ ]:
from datasets_asl import get_dataset
import torch
from torchvision import transforms as tf

# SETUP DATALOADER
dataset_test = get_dataset(
  **eval_cfg['dataset'],
  env = env_cfg,
  output_trafo = None
  )

dataloader_test = torch.utils.data.DataLoader(dataset_test,
  shuffle = False,
  num_workers = 0,
  pin_memory = True,
  batch_size = 2, 
  drop_last = True)

globale_idx_to_image_path = dataset_test.image_pths
from visu import Visualizer
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)

In [ ]:
from task import TaskCreator
from datasets_asl import get_dataloader_train, eval_lists_into_dataloaders

def get_mutiple_dataloaders(exp, env, train=True):
    tc = TaskCreator(**exp['task_generator'],output_size=exp['dataset']['output_size'])
    ret_list = []
    for t in tc:
        task, eval_lists = t
        dataloader_train, dataloader_buffer= get_dataloader_train(d_train= task.dataset_train_cfg,
                                                              env=env,exp = exp)
        if train: 
            ret_list.append( dataloader_train )
        else: 
            # RETURNS ALL VALIDATION DATALOADERS
            return eval_lists_into_dataloaders(eval_lists, env=env, exp=exp)
    return ret_list
        
dataloader_list = get_mutiple_dataloaders(exp = eval_cfg , env = env_cfg, train=False )

In [ ]:
model_paths = [ '/media/scratch1/jonfrey/models/cluster/2021-03-19T19:20:20_check_val_every_2_random/task0-epoch=29--step=016259.ckpt',
'/media/scratch1/jonfrey/models/cluster/2021-03-19T19:20:20_check_val_every_2_random/task1-epoch=63--step=030846.ckpt',
'/media/scratch1/jonfrey/models/cluster/2021-03-19T19:20:20_check_val_every_2_random/task2-epoch=95--step=045503.ckpt',
'/media/scratch1/jonfrey/models/cluster/2021-03-19T19:20:20_check_val_every_2_random/task3-epoch=135--step=060290.ckpt']
print( model_paths )
for f in dataloader_list:
    print(f.dataset)

In [ ]:
import copy

def get_grad( named_params ):
    summary_grad = {}
    for i, (n,p) in enumerate( named_params ):
        mod = n.split('.')[1]
        key = mod + '_weight' if n.find('conv.') != -1 else  mod + '_bias'    
        if key not in summary_grad.keys():
            summary_grad[key] = [torch.abs( p.grad.view(-1).detach() )]
        else:
            summary_grad[key].append( torch.abs(p.grad.view(-1).detach()) )
    for k in summary_grad.keys():
        summary_grad[k] = torch.cat( summary_grad[k] )

    return summary_grad

def get_grad_indidividual( named_params ):
    summary_grad = {}
    for i, (n,p) in enumerate( named_params ):
        summary_grad[n] = [torch.abs( p.grad.view(-1).detach() )]

    for k in summary_grad.keys():
        summary_grad[k] = torch.cat( summary_grad[k] )

    return summary_grad

## Welfords Online Algorithm
https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_online_algorithm

Online mean and variance estimation:

$
\bar x_n = \frac{(n-1) \, \bar x_{n-1} + x_n}{n} = \bar x_{n-1} + \frac{x_n - \bar x_{n-1}}{n} \!
$

$
{\displaystyle s_{n}^{2}={\frac {n-2}{n-1}}\,s_{n-1}^{2}+{\frac {(x_{n}-{\bar {x}}_{n-1})^{2}}{n}}=s_{n-1}^{2}+{\frac {(x_{n}-{\bar {x}}_{n-1})^{2}}{n}}-{\frac {s_{n-1}^{2}}{n-1}},\quad n>1}
{\displaystyle \sigma _{n}^{2}={\frac {(n-1)\,\sigma _{n-1}^{2}+(x_{n}-{\bar {x}}_{n-1})(x_{n}-{\bar {x}}_{n})}{n}}=\sigma _{n-1}^{2}+{\frac {(x_{n}-{\bar {x}}_{n-1})(x_{n}-{\bar {x}}_{n})-\sigma _{n-1}^{2}}{n}}.}
$

$
{\displaystyle \sigma _{n}^{2}={\frac {(n-1)\,\sigma _{n-1}^{2}+(x_{n}-{\bar {x}}_{n-1})(x_{n}-{\bar {x}}_{n})}{n}}=\sigma _{n-1}^{2}+{\frac {(x_{n}-{\bar {x}}_{n-1})(x_{n}-{\bar {x}}_{n})-\sigma _{n-1}^{2}}{n}}.}
$


In [ ]:
def t_update(existingAggregate, newValue):
    (count, mean, M2) = existingAggregate
    count += 1
    delta = newValue - mean
    if delta.isnan().any():
        print( "count", counte, " is nan")
    
    mean += delta / count
    delta2 = newValue - mean
    M2 += delta * delta2
    return (count, mean, M2)

# Retrieve the mean, variance and sample variance from an aggregate
def t_finalize(existingAggregate):
    (count, mean, M2) = existingAggregate
    if count < 2:
        return float("nan")
    else:
        (mean, variance, sampleVariance) = (mean, M2 / count, M2 / (count - 1))
        return (mean, variance, sampleVariance)

In [ ]:
def get_val_mean(dataloader_test, model, 
                 augmentation=True, nr_image_samples=1, accumulate_same_image= 50):
    mean = None
    count = {}
    trafo = tf.Compose([
      tf.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.05),
      tf.Normalize([.485, .456, .406], [.229, .224, .225])
    ])
    for i in range(accumulate_same_image):
        # SETUP DATALOADER
        for j, batch in enumerate( dataloader_test ):
#             try:
#                 print( )
#             except:

            
            
            # START EVALUATION  
            images = (batch[0]).cuda()
            target = batch[1].cuda()
            ori_img = batch[2]
            replayed = batch[3]
            BS = images.shape[0]
            global_idx = batch[4] 
            
            
#             print("AugRun: ", i, "Locale ID: ", j, "Gloable ID: ", global_idx, "Re: ", replayed, "faild:", target[target!=-1].sum())
            if augmentation:
                images = trafo(images)
                
            ret = model(images)
            
            loss = F.cross_entropy(ret[0], target, ignore_index=-1)
            torch.autograd.backward(loss)

            summary_grad = get_grad_indidividual( model.named_parameters() )

            # Apply Welford
            if mean is None:
                # INIT RUNNING MEAN AND VARIANCE
                mean = summary_grad 
                var = copy.deepcopy( summary_grad )
                count = { n:0 for n in summary_grad.keys()}
            else:
                if count[list( summary_grad.keys())[0]] == 1:
                    for k in var.keys():
                        var[k] = torch.abs( var[k] - summary_grad[k])  
                
                for k in var.keys():
                    # UPDATE RUNNING MEAN AND VARIANCE
                    count[k], mean[k], var[k] = t_update( (count[k], mean[k], var[k]), 
                                                             summary_grad[k]  )
            model.zero_grad()
            
            su = 0
            for m in mean.values():
                su += m.sum()
                
#             print( su )
            
            if j >= nr_image_samples:
                model.zero_grad()
                
                break

    sample_var = {}
    # Finalize Welford 
    
    for k in summary_grad.keys():
        mean[k], var[k], sample_var[k] = t_finalize( (count[k], mean[k], var[k]) )
    return mean, var

def plot_summary(var, mean, mean_only= True, var_only=False, name="" ):
    layer_aggregation_var = []
    layer_aggregation_mean = []
    for k in var.keys():
        layer_aggregation_var.append( var[k].mean() )
        layer_aggregation_mean.append( mean[k].mean() )
    if not mean_only and var_only:
        return visu.plot_bar( layer_aggregation_var , x_label= "Variance", y_label='Layerwise-Mean of Grad-Varaince over Sampeles', title="Overview all Layers" +name , sort=False, reverse=False, jupyter=False)
    return visu.plot_bar( layer_aggregation_mean , x_label= "Mean", y_label='Layerwise-Mean of Grad-Mean over Sampeles', title="Overview all Layers" + name , sort=False, reverse=False, jupyter=False)

    

In [ ]:
for d in dataloader_list:
    print(len(d),len( d.dataset)) 

In [ ]:
for jj in range(4):
    imgs = []
    for i in range(4):
        model = load(model,model_paths[i])
        print( model_paths[i], "batches" , len(dataloader_list[jj]) )
        var, mean = get_val_mean( dataloader_test= dataloader_list[jj], model = model, 
                                 augmentation=True, nr_image_samples=999, accumulate_same_image= 1)
        imgs.append( plot_summary(var, mean, mean_only= False, var_only=True, name= f'Samples from Task {jj}. Model Checkpoint Task {i}' ) )

    from torchvision.utils import make_grid
    imgs_torch = [torch.from_numpy(i) for i in imgs]
    imgs_torch = torch.stack( imgs_torch , dim = 0) 
    
    grid_images = make_grid(imgs_torch.permute(0,3,1,2), nrow = 2,padding = 2,
                  scale_each = False, pad_value = 0)
    visu.plot_image( grid_images, jupyter= True, store=True, tag=f'100_samples_fro_task{jj}_with_augmentation_')

In [ ]:
for k, v in model.named_parameters():
    print(k)

In [ ]:
grid_images.shape

In [ ]:
k = "classifier_weight"
visu.plot_bar( mean[k][::100] , x_label= k, y_label='Mean Value', title=k, sort=False, reverse=False, jupyter=True)

k = "extractor_bias"
visu.plot_bar( mean[k][::100] , x_label= k, y_label='Mean Value', title=k, sort=False, reverse=False, jupyter=True)


In [ ]:
mean[k][::100].shape

In [ ]:
def test_welfords_online_algorithm():
    # For a new value newValue, compute the new count, new mean, the new M2.
    # mean accumulates the mean of the entire dataset
    # M2 aggregates the squared distance from the mean
    # count aggregates the number of samples seen so far
    def update(existingAggregate, newValue):
        (count, mean, M2) = existingAggregate
        count += 1
        delta = newValue - mean
        mean += delta / count
        delta2 = newValue - mean
        M2 += delta * delta2
        return (count, mean, M2)

    # Retrieve the mean, variance and sample variance from an aggregate
    def finalize(existingAggregate):
        (count, mean, M2) = existingAggregate
        if count < 2:
            return float("nan")
        else:
            (mean, variance, sampleVariance) = (mean, M2 / count, M2 / (count - 1))
            return (mean, variance, sampleVariance)

    rva = np.random.randint( 0,100, (1000) )
    rv = rva.tolist()
    a = update( (0, 0, 0), 10 )

    for v in rv[1:]:
        a = update( a, v)
    a = finalize( a )
    print(a)
    print( rva.mean(), rva.std()**2)